In [ ]:
import sys
sys.path.append("..") # for sibling import

import compyute as cp

# Example 3.2

### Deep neural network using a custom model

The goal of this model is to classify iris species based on numerical features.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/uciml/iris and place it into the *data* directory.

In [ ]:
# ! pip install pandas

In [ ]:
import pandas as pd


data_orig = pd.read_csv('../data/iris.csv')
data = data_orig.copy()
data.drop(columns=['Id'], inplace=True)

data["Species"] = data["Species"].astype("category").cat.codes

data_tensor = cp.tensor(data.to_numpy())
train, val, test = cp.preprocessing.split_train_val_test(data_tensor)

X_train, y_train = train[:, :-1], train[:, -1].int()
X_val, y_val = val[:, :-1], val[:, -1].int()
X_test, y_test = test[:, :-1], test[:, -1].int()

X_train = cp.preprocessing.normalize(X_train, axis=0)
X_val = cp.preprocessing.normalize(X_val, axis=0)
X_test = cp.preprocessing.normalize(X_test, axis=0)

print (f'{X_train.shape=}')
print (f'{y_train.shape=}')

print (f'{X_val.shape=}')
print (f'{y_val.shape=}')

print (f'{X_test.shape=}')
print (f'{y_test.shape=}')

### Step 2: Build a custom model
If you want to define very specifically what the forward and backward pass of your model should look like, you can build your own custom model instead of using the predefined sequential model.

In [ ]:
import compyute.nn as nn
from compyute.nn.layers import *
from compyute.nn.inits import *

n_hidden = 100

class MyCustomModel(nn.Model):
    def __init__(self):
        super().__init__()

        # define your layers
        self.lin1 = Linear(4, n_hidden)
        self.tanh1 = Tanh()
        self.lin2 = Linear(n_hidden, n_hidden)
        self.tanh2 = Tanh()
        self.lin3 = Linear(n_hidden, 3)

    def forward(self, x):

        # define the forward pass
        y = self.tanh1(self.lin1(x))
        y = self.tanh2(self.lin2(y))
        y = self.lin3(y)

        # define the backward pass
        def backward(dy):
            tmp = self.lin3.backward(dy)
            tmp = self.lin2.backward(self.tanh2.backward(tmp))
            return self.lin1.backward(self.tanh1.backward(tmp))     
        self.backward = backward
        
        return y

model = MyCustomModel()

In [ ]:
model.compile(
    optimizer=nn.optimizers.SGD(),
    loss_fn=nn.losses.Crossentropy(),
    metric_fn=nn.metrics.accuracy
)

In [ ]:
from compyute.nn.analysis import model_summary
model_summary(model, (4,))

### Step 3: Train the model

In [ ]:
batch_size = len(X_train)

train_losses, train_scores, val_losses, val_scores = model.train(
    X_train, y_train, epochs=1000, val_data=(X_val, y_val), verbose=False, batch_size=batch_size
)

In [ ]:
traces = {
    "train_losses" : train_losses,
    "train_scores" : train_scores
}

nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="train history", x_label="steps", y_label="loss/accuracy")

In [ ]:
traces = {
    "val_losses" : val_losses,
    "val_scores" : val_scores
}

nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="val history", x_label="steps", y_label="loss/accuracy")

### Step 4: Evaluate the model
Using the defined metric, the model's performance can be evaluated using testing/validation data.

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size)
print(f'loss {loss:.4f}')
print(f'accuracy {100*accuracy:.2f}')